# Datas

> this notebook retrives at first all the edita that has been classified as weaponisation by the LLM and by Hamest

In [2]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Path to the folder containing CSVs or .xlsx
csv_folder = Path("datas/previous-project/Hedi/LLM Results")
hamest_file = Path("datas/previous-project/Hedi/hamest.xlsx")

# Create a list of DataFrames, one for each CSV
dfs = [pd.read_csv(f) for f in csv_folder.glob("*.csv")]
hamest = pd.read_excel(hamest_file)

dfs_named = {f.stem: pd.read_csv(f) for f in csv_folder.glob("*.csv")}

# Example: show how many DataFrames were loaded
print(f"Loaded {len(dfs)} CSV files.")
print("Available keys:", list(dfs_named.keys()))

Loaded 120 CSV files.
Available keys: ['COVID-19_pandemic_in_Ukraine_analysis', 'Immigration_to_Ukraine_analysis', 'Andrei_Sheptytsky_analysis', 'Football_in_Ukraine_analysis', 'Abortion_in_Ukraine_analysis', '2004_Ukrainian_presidential_election_analysis', 'History_of_Ukraine_analysis', 'Christianity_in_Russia_analysis', 'Cossack’s_songs_of_Dnipropetrovsk_Region_analysis', 'Flag_of_Ukraine_analysis', 'Belarusians_in_Ukraine_analysis', 'Cinema_of_Ukraine_analysis', 'Brotherhood_of_Independent_Baptist_Churches_and_Ministries_of_Ukraine_analysis', 'Hillsong_Ukraine_analysis', 'Communist_Party_of_the_Soviet_Union_analysis', 'Electricity_in_Ukraine_analysis', 'Censuses_in_Ukraine_analysis', 'Declaration_of_Independence_of_Ukraine_analysis', 'History_of_Christianity_in_Ukraine_analysis', 'Bessarabia_analysis', 'Human_trafficking_in_Ukraine_analysis', 'Crimea_analysis', 'Epiphanius_I_of_Ukraine_analysis', 'Hillsong_Church_Kiev_analysis', 'Government_of_the_Ukrainian_People_s_Republic_in_exil

In [6]:
dfs_named["2004_Ukrainian_presidential_election_analysis"].sample()
dfs_named["Abortion_in_Ukraine_analysis"].shape

(47, 7)

In [7]:
lines = 0
for df in dfs:
    lines += df.shape[0]
print(lines)

117516


In [8]:
# dfs_weaponised is a list of the dataframes with only edits that were classified as weaponised by the LLM
dfs_weaponised = [df[df['weaponised'] == 'Weaponised'] for df in dfs]

In [9]:
lines = 0
for df in dfs_weaponised:
    lines += df.shape[0]
print(lines)
# We have now 18707 rows that are classfied as weaponsied --> 18707 potential users.

18707


In [10]:
dfs_weaponised[0].sample()

,initial_version,changed_version,comment,user,date,llm_output,weaponised
1665,{{main|2020 coronavirus outbreak in Europe}}\n...,"--- \n+++ \n@@ -170,7 +170,7 @@\n \n On 7 Janu...",/* Vaccination */ En dash,EmmaCoop,2022-09-25T03:31:20Z,Change: Rephrased a sentence for clarity regar...,Weaponised


In [11]:
hamest_new = hamest.iloc[:, :-2] # Deleting the two last useless columns
hamest_new = hamest_new.rename(columns={"Annotation_1": "weaponised"}) # Rename last column
hamest_new_weaponised = hamest_new[hamest_new['weaponised'] == 'Weaponised']
hamest_new_weaponised.shape # (122, 5)
hamest_new_weaponised.head()

,Source,JSON_Record,Analysis,Judgment,weaponised
0,Kyiv_Pechersk_Lavra.jsonl,"{""version"": ""diff"", ""Timestamp"": ""2019-12-10T1...",Record 594 (Version: diff):\n**Judgment: Weapo...,Weaponised,Weaponised
3,Kyiv_Pechersk_Lavra.jsonl,"{""version"": ""diff"", ""Timestamp"": ""2020-09-18T1...",Record 646 (Version: diff):\n**Judgment: Weapo...,Weaponised,Weaponised
4,History_of_Christianity_in_Ukraine.jsonl,"{""version"": ""diff"", ""Timestamp"": ""2018-04-17T2...",Record 956 (Version: diff):\n**Judgment: Weapo...,Weaponised,Weaponised
5,Talk_History_of_Christianity_in_Ukraine.jsonl,"{""version"": ""diff"", ""Timestamp"": ""2007-03-05T0...",Record 291 (Version: diff):\n**Judgment: Weapo...,Weaponised,Weaponised
9,History_of_Christianity_in_Ukraine.jsonl,"{""version"": ""diff"", ""Timestamp"": ""2006-03-07T0...",Record 141 (Version: diff):\n**Judgment: Weapo...,Weaponised,Weaponised


In [13]:
import json

hamest_users = [
    json.loads(record).get("User")
    for record in hamest_new_weaponised["JSON_Record"]
    if pd.notna(record)
]

print(hamest_users[:20])

['Nepsis2', 'Materialscientist', 'Aleksandr Grigoryev', 'Irpen', 'Ukrained', 'Jumbo T', '119.94.109.231', 'Truthseeker 85.5', 'Irpen', 'Blobmob', 'Sirfurboy', 'Ушкуйник', 'Kuban kazak', 'Kuban kazak', 'USchick', 'UncleDouggie', '217.66.159.71', 'Kuban kazak', 'ClueBot', '79.66.27.98']


## Details about Wikipedia's users 

- User can be an IP adress --> IP users, guest users, or unregistered users are users who edit Wikipedia without registering for an account.
- User can be a Bot --> automatic editing
- User can be a registred user --> nickname or full name

### Bots

Let's focus on Bot at first because it feels weird that bots can edit Wikipedia's article. Apparently, bots are registered and approved. They must :
* Have a separate, clearly labeled account (usually ending with “Bot”, like ClueBot NG).
* Get approval from the community through the “Bot Approvals Group” process.
* Be limited in scope to well-defined, repetitive tasks (like fixing typos, reverting vandalism, adding categories, or updating statistics).
So, bots don’t randomly modify articles — they work under strict rules.

In [15]:
hamest_bot_users = [u for u in hamest_users if u and "bot" in u.lower()]
print(hamest_bot_users)

['ClueBot', 'Yobot', 'RMCD bot']


In [23]:
import ipaddress

hamest_ip_users = []
for u in hamest_users:
    try:
        if u:  # skip None
            ipaddress.ip_address(u)  # will raise ValueError if not an IP
            hamest_ip_users.append(u)
    except ValueError:
        continue

print(hamest_ip_users)

['119.94.109.231', '217.66.159.71', '79.66.27.98', '46.211.108.39', '2600:1702:4630:7060:4158:952D:5639:86F6', '117.242.204.179', '219.166.46.101', '95.132.51.19', '77.87.152.7', '72.28.148.137', '91.221.145.225', '94.192.125.190', '71.202.160.171', '84.9.227.23', '134.84.5.47', '69.119.175.240', '71.70.224.198', '72.93.158.40', '81.211.44.171', '95.133.130.135', '109.71.177.2', '24.97.197.161', '219.166.46.101', '70.26.84.3', '70.52.61.104', '50.46.234.37', '49.205.32.28']


In [25]:
bot_indices = [
    ind
    for ind, val in enumerate(hamest_users)
    if val and "bot" in val.lower()
]

ip_indices = []
for ind, val in enumerate(hamest_users):
    try:
        if val:  # skip None
            ipaddress.ip_address(val)  # throws if not valid IP
            ip_indices.append(ind)
    except ValueError:
        continue

print(ip_indices)

[6, 16, 19, 20, 26, 35, 46, 47, 51, 52, 53, 55, 56, 58, 61, 66, 67, 68, 73, 81, 92, 93, 96, 102, 107, 108, 119]


In [21]:
for ind in bot_indices:
    print(hamest_new_weaponised.iloc[ind]['JSON_Record'])
    print()

{"version": "diff", "Timestamp": "2009-05-21T01:06:15Z", "User": "ClueBot", "Comment": "Reverting possible vandalism by [[Special:Contributions/Ukrainomanihate|Ukrainomanihate]] to version by Rholton. False positive? [[User:ClueBot/FalsePositives|Report it]]. Thanks, [[User:ClueBot|ClueBot]]. (707240) (Bot)", "Diff": "--- \n+++ \n@@ -1,7 +1,7 @@\n [[Image:Virsky First Dance.JPG|thumb|300px|right|A performance of a traditional Ukrainian dance by [[Virsky]] dance ensemble]]\n {{Ukrainians}}\n \n-The '''Culture of [[Ukraine]]''' is a result of influence over millennia from the West and East for being big homosexuals, with an assortment of strong culturally-identified [[ethnic]] groups, woh likes to suck on big male genetales until the penis ejaculates. Like most [[Western world|Western]] countries, Ukrainian customs are heavily influenced by [[Christianity]]. [[Russia]]n and other Eastern European cultures also have had a more or less significant impact on the Ukrainian culture.\n+The '''

In [22]:
threats = []
threats.append('Ukrainomanihate')

In [26]:
for ind in ip_indices[::5]:
    print(hamest_new_weaponised.iloc[ind]['JSON_Record'])
    print()

{"version": "diff", "Timestamp": "2018-04-11T12:06:25Z", "User": "119.94.109.231", "Comment": "", "Diff": "--- \n+++ \n@@ -1,5 +1,7 @@\n {{Culture of Ukraine}}\n-'''Ukrainian culture''' and customs of [[Ukraine]] and ethnic [[Ukrainians]]. Ukraine has a shared culture and history with neighboring nations, dating back to the 9th century and the [[Rus (name)|Land of Rus]]. Mutual customs are shared among the cultures of  [[Ukraine]], [[Culture of Belarus|Belarus]], [[Ukrainian Cossacks]] and [[Russians]]. Ukrainian customs are heavily influenced by the [[Eastern Orthodox Church]] and traditions from [[Slavic mythology]]. The [[Soviet era]] unified the cultures of many unrelated nations with a common language.\n+'''Ukrainian culture''' \n+are the stupid\n+ customs of [[Ukraine]] and ethnic [[Ukrainians]]. Ukraine has a shared culture and history with neighboring nations, dating back to the 9th century and the [[Rus (name)|Land of Rus]]. Mutual customs are shared among the cultures of  [[U